# 1. Sobre o Projeto e Referências

## Indicadores

Com base no dicionário de dados fornecido pela Passos Mágicos, ficou definido que os seguintes indicadores serão utilizados:

### **1. Desempenho em Disciplinas Específicas**  
- **Dados necessários:** `NOTA_PORT`, `NOTA_MAT`, `NOTA_ING`.  
- **Descrição:** Médias de notas em disciplinas específicas, como português, matemática e inglês.

### **2. Frequência Escolar e Engajamento**  
- **Dados necessários:** `FASE`, `TURMA`.  
- **Descrição:** Dados sobre a fase e turma dos alunos, que podem ser utilizados para medir frequência e engajamento ao longo dos anos.

### **3. Indicador de Desempenho Acadêmico (INDE)**  
- **Dados necessários:** `INDE`.  
- **Descrição:** O Índice do Desenvolvimento Educacional (INDE) é uma métrica que combina outros indicadores (IAN, IDA, IEG, IAA, IPS, IPP e IPV) para avaliar o desempenho acadêmico geral dos alunos ao longo dos anos.

### **4. Indicador de Aprendizagem (IDA)**  
- **Dados necessários:** `IDA`.  
- **Descrição:** Média das notas do indicador de aprendizagem, medindo o progresso acadêmico dos alunos.

### **5. Indicador de Engajamento (IEG)**  
- **Dados necessários:** `IEG`.  
- **Descrição:** Média das notas de engajamento dos alunos, mensurando a participação e envolvimento deles nas atividades.

### **6. Indicador Psicossocial (IPS)**  
- **Dados necessários:** `IPS`.  
- **Descrição:** Média das notas psicossociais dos alunos, avaliando seu bem-estar emocional e social.

### **7. Indicador Psicopedagógico (IPP)**  
- **Dados necessários:** `IPP`.  
- **Descrição:** Média das notas psicopedagógicas dos alunos, refletindo o suporte que receberam nesse aspecto.

### **8. Indicador de Autoavaliação (IAA)**  
- **Dados necessários:** `IAA`.  
- **Descrição:** Média das notas de autoavaliação dos alunos, que refletem sua percepção de progresso.

### **9. Indicador de Ponto de Virada (IPV)**  
- **Dados necessários:** `IPV`.  
- **Descrição:** Média das notas que indicam o "ponto de virada" — um momento de mudança significativa no desempenho ou comportamento do aluno.

### **10. Classificação de Desempenho (Pedra)**  
- **Dados necessários:** `PEDRA`.  
- **Descrição:** Classificação dos alunos com base no seu INDE, em níveis como Quartzo, Ágata, Ametista e Topázio.

### **11. Recomendações Psicopedagógicas**  
- **Dados necessários:** `REC_PSICO`.  
- **Descrição:** Recomendações feitas pela equipe psicopedagógica para acompanhamento e suporte ao aluno.

---

## 2.1 Tabela de Indicadores Sugeridos  

| Nome do Campo             | Descrição                                                                                                                                       | Tipo de Dados |
|---------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------|---------------|
| `NOTA_PORT`, `NOTA_MAT`, `NOTA_ING` | Médias de notas em disciplinas específicas, como português, matemática e inglês.                                                            | Numérico      |
| `FASE`, `TURMA`           | Dados sobre a fase e turma dos alunos, que podem ser utilizados para medir frequência e engajamento ao longo dos anos.                            | Texto         |
| `INDE`                    | O Índice do Desenvolvimento Educacional (INDE) é uma métrica que combina outros indicadores (IAN, IDA, IEG, IAA, IPS, IPP e IPV) para avaliar o desempenho acadêmico geral dos alunos ao longo dos anos. | Numérico      |
| `IDA`                     | Média das notas do indicador de aprendizagem, medindo o progresso acadêmico dos alunos.                                                          | Numérico      |
| `IEG`                     | Média das notas de engajamento dos alunos, mensurando a participação e envolvimento deles nas atividades.                                        | Numérico      |
| `IPS`                     | Média das notas psicossociais dos alunos, avaliando seu bem-estar emocional e social.                                                            | Numérico      |
| `IPP`                     | Média das notas psicopedagógicas dos alunos, refletindo o suporte que receberam nesse aspecto.                                                   | Numérico      |
| `IAA`                     | Média das notas de autoavaliação dos alunos, que refletem sua percepção de progresso.                                                            | Numérico      |
| `IPV`                     | Média das notas que indicam o "ponto de virada" — um momento de mudança significativa no desempenho ou comportamento do aluno.                    | Numérico      |
| `PEDRA`                   | Classificação dos alunos com base no seu INDE, em níveis como Quartzo, Ágata, Ametista e Topázio.                                                | Texto         |
| `REC_PSICO`               | Recomendações feitas pela equipe psicopedagógica para acompanhamento e suporte ao aluno.                                                          | Texto         |


# 3. Análise e Desenvolvimento do DataFrame

In [ ]:
# %pip install pandas
# %pip install pyspark
# %pip install seaborn

In [1]:
# importação das bibliotecas que serão utilizadas
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from pyspark.sql import SparkSession
import os

In [5]:
class SparkPostgres:
    def __init__(self, spark, host, port, database, user, password, schema=None):
        self.spark = spark
        self.url = f"jdbc:postgresql://{host}:{port}/{database}"
        self.schema = schema
        self.properties = {
            "user": user,
            "password": password,
            "driver": "org.postgresql.Driver"
        }

    def read(self, table):
        # Adiciona o esquema, se definido
        full_table = f"{self.schema}.{table}" if self.schema else table
        return self.spark.read.jdbc(url=self.url, table=full_table, properties=self.properties)

    def create_temp_view(self, sql_query, view_name):
        # Inclui o esquema em tabelas na consulta SQL
        if self.schema:
            sql_query = self._add_schema_to_query(sql_query)

        df = self.spark.read.jdbc(url=self.url, table=f"({sql_query}) as tmp", properties=self.properties)
        df.createOrReplaceTempView(view_name)
        return df

    def _add_schema_to_query(self, sql_query):
        """Adiciona o esquema a todas as tabelas da consulta SQL."""
        # Substitui nomes de tabela simples pelo esquema e tabela, assumindo que não há alias complexos
        import re
        pattern = r'\bFROM\s+(\w+)'
        return re.sub(pattern, fr'FROM {self.schema}.\1', sql_query, flags=re.IGNORECASE)


In [8]:
spark = SparkSession.builder.appName("CSV to PostgreSQL").getOrCreate()

host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
database = os.getenv("DB_DATABASE")
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
schema = os.getenv("DB_SCHEMA")

spark_postgres = SparkPostgres(spark, host, port, database, user, password, schema)

sql_query = """
SELECT
    CAST(nome AS TEXT) AS nome,
    CAST(instituicao_ensino_aluno AS TEXT) AS instituicao_ensino_aluno,
    CAST(idade_aluno AS TEXT) AS idade_aluno,
    CAST(anos_pm AS TEXT) AS anos_pm,
    CAST(fase_turma AS TEXT) AS fase,
    CAST(ponto_virada AS TEXT) AS ponto_virada,
    CAST(inde AS TEXT) AS inde,
    CAST(inde_conceito AS TEXT) AS inde_conceito,
    CAST(pedra AS TEXT) AS pedra,
    CAST(destaque_ieg AS TEXT) AS destaque_ieg,
    CAST(destaque_ida AS TEXT) AS destaque_ida,
    CAST(destaque_ipv AS TEXT) AS destaque_ipv,
    CAST(iaa AS TEXT) AS iaa,
    CAST(ieg AS TEXT) AS ieg,
    CAST(ips AS TEXT) AS ips,
    CAST(ida AS TEXT) AS ida,
    CAST(ipp AS TEXT) AS ipp,
    CAST(ipv AS TEXT) AS ipv,
    CAST(ian AS TEXT) AS ian,
    CAST(ano AS TEXT) AS ano,
    CAST(dt_pst AS TEXT) AS dt_pst
FROM
    tb_dados_principais_2020

UNION ALL

SELECT
    CAST(nome AS TEXT) AS nome,
    CAST(instituicao_ensino_aluno AS TEXT) AS instituicao_ensino_aluno,
    CAST(NULL AS TEXT) AS idade_aluno,
    CAST(NULL AS TEXT) AS anos_pm,
    CAST(fase AS TEXT) AS fase,
    CAST(ponto_virada AS TEXT) AS ponto_virada,
    CAST(inde AS TEXT) AS inde,
    CAST(NULL AS TEXT) AS inde_conceito,
    CAST(pedra AS TEXT) AS pedra,
    CAST(NULL AS TEXT) AS destaque_ieg,
    CAST(NULL AS TEXT) AS destaque_ida,
    CAST(NULL AS TEXT) AS destaque_ipv,
    CAST(iaa AS TEXT) AS iaa,
    CAST(ieg AS TEXT) AS ieg,
    CAST(ips AS TEXT) AS ips,
    CAST(ida AS TEXT) AS ida,
    CAST(ipp AS TEXT) AS ipp,
    CAST(ipv AS TEXT) AS ipv,
    CAST(ian AS TEXT) AS ian,
    CAST(ano AS TEXT) AS ano,
    CAST(dt_pst AS TEXT) AS dt_pst
FROM
    tb_dados_principais_2021

UNION ALL

SELECT
    CAST(nome AS TEXT) AS nome,
    CAST(NULL AS TEXT) AS instituicao_ensino_aluno,
    CAST(NULL AS TEXT) AS idade_aluno,
    CAST(NULL AS TEXT) AS anos_pm,
    CAST(fase AS TEXT) AS fase,
    CAST(ponto_virada AS TEXT) AS ponto_virada,
    CAST(inde AS TEXT) AS inde,
    CAST(NULL AS TEXT) AS inde_conceito,
    CAST(pedra AS TEXT) AS pedra,
    CAST(destaque_ieg AS TEXT) AS destaque_ieg,
    CAST(destaque_ida AS TEXT) AS destaque_ida,
    CAST(destaque_ipv AS TEXT) AS destaque_ipv,
    CAST(iaa AS TEXT) AS iaa,
    CAST(ieg AS TEXT) AS ieg,
    CAST(ips AS TEXT) AS ips,
    CAST(ida AS TEXT) AS ida,
    CAST(ipp AS TEXT) AS ipp,
    CAST(ipv AS TEXT) AS ipv,
    CAST(ian AS TEXT) AS ian,
    CAST(ano AS TEXT) AS ano,
    CAST(dt_pst AS TEXT) AS dt_pst
FROM
    tb_dados_principais_2022



    """

df_dados_principais_spark = spark_postgres.create_temp_view(sql_query, "tb_dados_principais")
df_aluno_spark = spark_postgres.read("tb_aluno_mrg")
df_historico_spark = spark_postgres.read("tb_historico_mrg")
df_fase_spark = spark_postgres.read("tb_fase_mrg")

In [13]:
# df_dados_principais_spark.write.format("csv").option("header", "true").save("dados_principais")
df_aluno_spark.write.format("csv").option("header", "true").save("aluno")
df_historico_spark.write.format("csv").option("header", "true").save("historico")
df_fase_spark.write.format("csv").option("header", "true").save("fase")

25/02/09 18:34:21 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [ ]:
df_dados_principais = df_dados_principais_spark.toPandas()
df_aluno = df_aluno_spark.toPandas()
df_historico = df_historico_spark.toPandas()
df_fase = df_fase_spark.toPandas()

In [ ]:
df_pm = pd.read_csv('bases_e_documental\pede_passos_dataset_fiap.csv', sep=';')
pd.set_option('display.max_columns', None)
df_pm

In [ ]:
df_alunos = pd.read_csv('bases_e_documental/Base de dados - Passos Mágicos/TbAluno/Merge/merged_data.csv', sep=',')
df_fase = pd.read_csv('bases_e_documental/Base de dados - Passos Mágicos/TbFase/Merge/merged_data.csv', sep=',')
df_historico = pd.read_csv('bases_e_documental\Base de dados - Passos Mágicos\TbHistorico\Merge\merged_data.csv', sep=',')

In [ ]:
# verificar se os três dataframes tem o IdAluno, para que possa ser utilizado como referência no merge
print('IdAluno' in df_alunos.columns)
print('IdAluno' in df_fase.columns)
print('IdAluno' in df_historico.columns)

In [ ]:
# junta os 3 dataframes em um só
# Realizar o merge dos DataFrames usando 'IdAluno' como referência
df_merged = df_alunos.merge(df_fase, on='IdAluno', how='left')
df_merged = df_merged.merge(df_historico, on='IdAluno', how='left')

In [ ]:
df_merged.head()

In [ ]:
df_merged.shape

In [ ]:
df_merged.info(verbose=True, buf=None)

In [ ]:
# verifica porcentagem de dados nulos por coluna
df_merged.isnull().sum()/df_merged.shape[0]*100

In [ ]:
# faz o drop da coluna, se 100% dos valores da coluna estiverem nulos
# identifica colunas com 100% de valores nulos
colunas_para_deletar = df_merged.columns[df_merged.isnull().all()]

# converte objeto para uma lista
colunas_para_deletar_lista = colunas_para_deletar.tolist()

# criar um DataFrame para exibir a lista
df_colunas_para_deletar = pd.DataFrame(colunas_para_deletar_lista, columns=['Colunas para Deletar'])

# adiciona uma linha com o total de colunas deletadas
total_deletadas = len(colunas_para_deletar_lista)
df_colunas_para_deletar.loc['Total'] = [f'Total de colunas deletadas: {total_deletadas}']

# exibe o DataFrame resultante
df_colunas_para_deletar

In [ ]:
# deleta as colunas identificadas
df_merged.drop(columns=colunas_para_deletar, inplace=True)

# Exibir o DataFrame atualizado
df_merged.head()

In [ ]:
# mostra dataframe inteiro, sem truncar informações
pd.set_option('display.max_columns', None)
df_merged.head()

In [ ]:
# altera informações da coluna IdTipoResponsável para facilitar a visualização
df_merged['IdTipoResponsavel'] = df_merged['IdTipoResponsavel'].replace({1 : 'Pai', 2 : 'Mãe', 3 : 'O Próprio', 4 : 'Outros', 5 : 'Tia', 6 : 'Padrasto', 7 : 'Madrasta', 8 : 'Avó', 9 : 'Avô', 10 : 'Tio', 11 : 'Não definido', 12 : 'Outros Responsáveis', 13 : 'Mãe', 14 : 'Irmão', 15 : 'IRM'})

In [ ]:
# contagem de valores de uma coluna
df_merged['IdTipoResponsavel'].value_counts()

In [ ]:
# altera informações da coluna Sexo para facilitar a visualização
df_merged['Sexo'] = df_merged['Sexo'].replace({'F': 'Feminino', 'M': 'Masculino'})

In [ ]:
# altera informações da coluna CoraRaca para facilitar a visualização
df_merged['CorRaca'] = df_merged['CorRaca'].replace({'B': 'Branca', 'P': 'Preta', 'R': 'Não especificado'})

In [ ]:
# verifica porcentagem de dados nulos por coluna
df_merged.isnull().sum()/df_merged.shape[0]*100

In [ ]:
# drop em colunas que possuem menos de 100% do total de valores nulos - analisado caso - a - caso antes da deleção
df_merged.drop(columns=['IdUsuarioEfetivacaoMatricula', 'ProblemaAutorizadoMatricula', 'IdUsuarioAutorizacaoMatricula', 'IdMotivoInativacao', 'IdDisciplina', 'StDependencia', 'NotaFinal', 'IdHistoricoNotas', 'StUsaCargaHorariaAnualHoraMinutoTexto', 'IdEstabelecimentoEnsino', 'StUsaCargaHorariaTotalHoraMinutoTexto', 'StCHIgnorarSoma_x', 'StCHIgnorarSoma_y'], inplace=True)  

In [ ]:
df_merged.shape

In [ ]:
# transforma colunas object do dataframe em datetime
columns = ['Data', 'DataOcorrencia_x', 'DataInclusao', 'DataSituacaoAtivo', 'DataSituacaoInativo', 'DataHoraEfetivacaoMatricula', 'DataNascimento', 'DataOcorrencia_y']
df_merged[columns] = df_merged[columns].apply(pd.to_datetime, errors='coerce')

In [ ]:
# Certifica que as colunas são do tipo datetime
df_merged['DataNascimento'] = pd.to_datetime(df_merged['DataNascimento'], errors='coerce')
df_merged['DataInclusao'] = pd.to_datetime(df_merged['DataInclusao'], errors='coerce')
df_merged['DataHoraEfetivacaoMatricula'] = pd.to_datetime(df_merged['DataHoraEfetivacaoMatricula'], errors='coerce')
df_merged['DataOcorrencia_y'] = pd.to_datetime(df_merged['DataOcorrencia_y'], errors='coerce')

In [ ]:
# transforma colunas datetime em data, sem horário
df_merged[['DataNascimento', 'DataInclusao', 'DataHoraEfetivacaoMatricula', 'DataOcorrencia_y']] = df_merged[['DataNascimento', 'DataInclusao', 'DataHoraEfetivacaoMatricula', 'DataOcorrencia_y']].apply(lambda x: x.dt.date)

In [ ]:
df_merged.info(verbose=True, buf=None)

In [ ]:
df_merged.head()

In [ ]:
df_merged.describe().T

In [ ]:

# contagem de valores nulos
null_values = df_merged.isnull().sum()

# calores únicos em cada coluna
unique_values = df_merged.nunique()

# análise descritiva básica
desc_stats = df_merged.describe(include='all')

# exibir as análises descritivas, contagem de valores nulos e valores únicos
null_values, unique_values, desc_stats

## 3.1 Contagem de Valores Nulos

A contagem de valores nulos nas principais colunas identificou um número significativo de valores faltantes em várias colunas importantes, incluindo:

- `QuantidadeFaltasAnual`, `ResultadoFinal_x`, `CodigoSerie_y`, e `AnoConclusao_y`: Todas apresentam cerca de 32.714 valores nulos em 49.089 registros.

- Isso representa quase 67% de valores nulos nessas colunas, o que é um ponto crítico que pode comprometer as análises de desempenho, engajamento e progresso acadêmico.

**Implicações:**

- `QuantidadeFaltasAnual`, `ResultadoFinal_x`, `CodigoSerie_y`, e `AnoConclusao_y`: Todas apresentam cerca de 32.714 valores nulos em 49.089 registros.

- Isso representa quase 67% de valores nulos nessas colunas, o que é um ponto crítico que pode comprometer as análises de desempenho, engajamento e progresso acadêmico.

## 3.2 Valores Únicos

Algumas colunas possuem valores extremamente repetitivos ou até únicos:

- **`ResultadoFinal_x`, `CodigoSerie_y`, e `AnoConclusao_y`**: Cada um desses campos tem apenas um valor único em todos os registros.

- Para `ResultadoFinal_x`, todos os registros disponíveis estão marcados como "Aprovado". Isso levanta dúvidas sobre a integridade ou completude dos dados.

**Implicações:**

- **Dados de resultados limitados:** Com um único valor "Aprovado" para todos os registros, fica impossível distinguir entre alunos que passaram ou falharam. Isso sugere que há uma falta de dados sobre alunos que não tiveram bom desempenho.

- **Falta de variabilidade em séries e anos de conclusão:** Isso impede uma análise longitudinal, ou seja, uma análise que verifique melhorias ou mudanças ao longo do tempo. Se todos os alunos estão no mesmo ano ou série, é difícil entender se há um progresso significativo.

## 3.3 Análise Descritiva

A função `describe()` fornece algumas estatísticas básicas sobre a distribuição dos dados:

- **`IdAlunoRotinaEducacaoInfantil` e `IdAluno`**: O número de alunos é relativamente pequeno em comparação com o total de registros (49.089), o que indica que os mesmos alunos aparecem várias vezes.

- **Faltas Anuais**: A média registrada é zero, o que reforça a ausência de dados de faltas, sendo uma área de grande preocupação.

**Implicações:**

- **Dados repetidos por aluno:** O fato de que um pequeno número de alunos aparece repetidamente sugere que estamos lidando com dados de múltiplas observações por aluno (provavelmente ao longo do tempo ou em diferentes turmas), o que seria normal em um acompanhamento contínuo. No entanto, seria necessário limpar e organizar os dados para focar nas métricas mais significativas por aluno.

- **Faltas não registradas:** Com uma média de faltas igual a zero, conclui-se que os dados de frequência são ineficazes para análises que dependem de engajamento ou presença dos alunos.

## 3.4 Pontos de Atenção

- **Preenchimento de valores nulos:** A base de dados deve ser revisada para preencher os valores nulos ou lidar com eles de forma apropriada. Dependendo da disponibilidade de dados, pode ser necessário estimar alguns desses valores ou remover registros incompletos.

- **Registro de faltas e resultados finais:** Há uma necessidade crítica de melhorar o registro de informações sobre faltas e desempenho acadêmico. Sem essas informações, será impossível medir adequadamente o progresso dos alunos e o impacto da ONG.

- **Variabilidade dos dados:** A falta de variabilidade em dados importantes, como os resultados finais e séries, limita qualquer análise significativa. Será importante diversificar os dados coletados, especialmente em termos de resultados de alunos.

## 3.5 Recomendações para Melhorar a Análise

- **Revisar a coleta de dados**: É necessário garantir que todos os alunos tenham seus resultados finais e faltas registrados corretamente.

- **Tratar os dados ausentes**: Colunas com muitos valores nulos precisam ser preenchidas (se possível) ou desconsideradas caso estejam comprometendo as análises.

- **Analisar alunos individualmente**: Dado que existem múltiplos registros por aluno, seria útil realizar uma análise focada em cada aluno individualmente, para evitar a distorção causada por múltiplos registros que, talvez, não acrescentem valor.

## 3.6 Representação Visual dos Problemas de base

In [ ]:
# Gráfico 1: Quantidade de valores nulos por coluna
plt.figure(figsize=(12, 6))
null_values.plot(kind='bar', color='salmon')
plt.title('Quantidade de Valores Nulos por Coluna')
plt.ylabel('Quantidade de Valores Nulos')
plt.xlabel('Colunas')
plt.xticks(rotation=90)
plt.show()

# Gráfico 2: Distribuição dos Resultados Finais (ResultadoFinal_x)
plt.figure(figsize=(8, 5))
df_merged['ResultadoFinal_x'].fillna('Não informado').value_counts().plot(kind='bar', color='lightblue')
plt.title('Distribuição dos Resultados Finais')
plt.ylabel('Contagem')
plt.xlabel('Resultado Final')
plt.show()

# Gráfico 3: Histograma da Distribuição de Faltas Anuais
plt.figure(figsize=(8, 5))
sns.histplot(df_merged['QuantidadeFaltasAnual'].fillna(0), bins=30, kde=True, color='skyblue')
plt.title('Distribuição da Quantidade de Faltas Anual')
plt.xlabel('Faltas Anuais')
plt.show()

- **Quantidade de Valores Nulos por Coluna**: Número de valores ausentes em cada coluna, com destaque para colunas críticas como `QuantidadeFaltasAnual`, `ResultadoFinal_x`, e `AnoConclusao_y`, que apresentam muitos valores nulos.

- `Distribuição dos Resultados Finais`: A maioria dos registros está classificada como "Aprovado", com uma grande parte de dados ausentes (preenchidos como "Não informado"), indicando um problema de variabilidade nos dados de desempenho.

- `Distribuição da Quantidade de Faltas Anual`: Embora muitos dados estejam faltando, o histograma mostra que a maioria dos alunos não tem faltas registradas, o que levanta a questão sobre a completude dos dados de presença.

## 3.7 Conclusão sobre a Viabilidade da Base de Dados

Os dados parecem incompletos em várias áreas críticas, como o desempenho acadêmico e a presença dos alunos. Além disso, a maioria dos alunos está marcada como "Aprovado", o que pode refletir uma simplificação excessiva ou a ausência de registros de reprovação, distorcendo os resultados.

Se os ajustes propostos forem implementados com sucesso (preenchimento de nulos, maior variabilidade nos resultados e dados completos), a base pode ser viável, mas limitada. Certas análises (como de presença e engajamento) ainda seriam prejudicadas sem dados adequados.

Para a base compartilhada, a análise ser focada nos dados que estão mais completos, como análises específicas de desempenho com os alunos que possuem dados completos.

## 3.8 Recomendações Finais
Focar em Subconjuntos de Dados Completos: Muitas das análises viáveis dependem de filtrar o dataset para trabalhar apenas com os registros que possuem dados completos. Embora isso reduza o tamanho da amostra, ainda é possível gerar insights importantes.

Documentar Limitações: Ao realizar qualquer uma dessas análises, é crucial documentar as limitações impostas pelos dados ausentes. Isso garante que as conclusões sejam interpretadas com cautela e que os resultados não sejam extrapolados indevidamente.

Melhorar a Coleta de Dados Futuramente: Embora possamos realizar análises viáveis com o que temos, é fundamental melhorar a coleta de dados no futuro, especialmente em áreas críticas como desempenho acadêmico, frequência e registros socioeconômicos.

## 3.9 Tratamento da Base Municipios

In [ ]:
df_municipio = pd.read_csv('bases_e_documental/Base de dados - Passos Mágicos/Outras tabelas/TbMunicipio.csv', sep=',')
df_municipio.head()

In [ ]:
df_municipio.shape

In [ ]:
df_municipio.info()

In [ ]:
df_municipio.isnull().sum()

In [ ]:
# exporta a base de municipios para csv
# df_municipio.to_csv('bases_e_documental/df_municipio.csv', index=False)

# 4. Gráficos de Apoio para o Relatório Final

## 4.1 Análise de Desempenho dos Alunos com Dados Completos

In [ ]:
# Filtrar os dados para incluir apenas os registros com dados completos para ResultadoFinal_x, AnoConclusao_y, e QuantidadeFaltasAnual
df_filtered = df_merged.dropna(subset=['ResultadoFinal_x', 'AnoConclusao_y', 'QuantidadeFaltasAnual'])

# Contar o número de alunos aprovados (ResultadoFinal_x = 'Aprovado') e reprovados
aprovados = df_filtered['ResultadoFinal_x'].value_counts()

# Calcular a taxa de aprovação
total_alunos = len(df_filtered)
taxa_aprovacao = aprovados.get('A', 0) / total_alunos if total_alunos > 0 else 0

# Exibir a taxa de aprovação e o total de alunos
print(f"Taxa de Aprovação: {taxa_aprovacao * 100}%")
print(f"Total de Alunos com Dados Completos: {total_alunos}")

In [ ]:
# Gerar gráfico de barras para visualização do desempenho
plt.figure(figsize=(8, 5))
sns.countplot(x='ResultadoFinal_x', data=df_filtered, palette='Blues')
plt.title('Distribuição de Resultados Finais dos Alunos com Dados Completos')
plt.xlabel('Resultado Final')
plt.ylabel('Contagem de Alunos')
plt.show()

## 4.2 Correlação entre Variáveis Socioeconômicas e Desempenho

In [ ]:
# Filtrar os dados para incluir registros com dados completos para ResultadoFinal_x e IdResponsavelFinanceiro
df_filtered_socio = df_filtered.dropna(subset=['IdResponsavelFinanceiro'])

# Converter a coluna de resultado final para uma variável binária (Aprovado = 1, Reprovado = 0)
df_filtered_socio['Aprovado'] = df_filtered_socio['ResultadoFinal_x'].apply(lambda x: 1 if x == 'A' else 0)

# Calcular a correlação entre IdResponsavelFinanceiro e o resultado acadêmico (Aprovado)
correlacao_socio_desempenho = df_filtered_socio['IdResponsavelFinanceiro'].corr(df_filtered_socio['Aprovado'])

# Exibir a correlação calculada
print(f"Correlação entre IdResponsavelFinanceiro e Aprovado: {correlacao_socio_desempenho}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Gerar um gráfico de dispersão para visualizar a relação entre IdResponsavelFinanceiro e Aprovado
plt.figure(figsize=(10, 6))
sns.scatterplot(x='IdResponsavelFinanceiro', y='Aprovado', data=df_filtered_socio, marker='x', color='blue')
plt.title('Correlação entre Status Socioeconômico (IdResponsavelFinanceiro) e Desempenho Acadêmico')
plt.xlabel('IdResponsavelFinanceiro')
plt.ylabel('Aprovado (1 = Aprovado, 0 = Reprovado)')
plt.show()

## 4.3 Análise de Engajamento e Faltas

In [ ]:
# Filtrar os dados para incluir registros com dados completos para QuantidadeFaltasAnual e ResultadoFinal_x
df_filtered_faltas = df_filtered.dropna(subset=['QuantidadeFaltasAnual', 'ResultadoFinal_x'])

# Converter a coluna de resultado final para uma variável binária (Aprovado = 1, Reprovado = 0)
df_filtered_faltas['Aprovado'] = df_filtered_faltas['ResultadoFinal_x'].apply(lambda x: 1 if x == 'A' else 0)

# Criar um grupo de alunos com baixas faltas (menor que a mediana) e altas faltas (maior ou igual à mediana)
mediana_faltas = df_filtered_faltas['QuantidadeFaltasAnual'].median()
df_filtered_faltas['GrupoFaltas'] = df_filtered_faltas['QuantidadeFaltasAnual'].apply(lambda x: 'Baixas Faltas' if x < mediana_faltas else 'Altas Faltas')

# Comparar o desempenho entre grupos de baixas e altas faltas
plt.figure(figsize=(10, 6))
sns.boxplot(x='GrupoFaltas', y='Aprovado', data=df_filtered_faltas)
plt.title('Comparação entre Faltas e Desempenho Acadêmico')
plt.xlabel('Grupo de Faltas')
plt.ylabel('Aprovado (1 = Aprovado, 0 = Reprovado)')
plt.show()

# Exibir a mediana das faltas para referência
print(f"Mediana de Faltas: {mediana_faltas}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Gráfico de comparação entre grupos de baixas e altas faltas no desempenho
plt.figure(figsize=(10, 6))
sns.boxplot(x='GrupoFaltas', y='Aprovado', data=df_filtered_faltas)
plt.title('Comparação entre Faltas e Desempenho Acadêmico')
plt.xlabel('Grupo de Faltas')
plt.ylabel('Aprovado (1 = Aprovado, 0 = Reprovado)')
plt.show()

## 4.4 Evolução Temporal do Desempenho

In [ ]:
# Filtrar os dados para incluir registros com dados completos para ResultadoFinal_x e AnoConclusao_y
df_filtered_ano = df_filtered.dropna(subset=['AnoConclusao_y', 'ResultadoFinal_x'])

# Converter a coluna de resultado final para uma variável binária (Aprovado = 1, Reprovado = 0)
df_filtered_ano['Aprovado'] = df_filtered_ano['ResultadoFinal_x'].apply(lambda x: 1 if x == 'A' else 0)

# Agrupar os dados por ano de conclusão e calcular a taxa de aprovação
taxa_aprovacao_por_ano = df_filtered_ano.groupby('AnoConclusao_y')['Aprovado'].mean()

# Exibir as taxas de aprovação por ano
print(taxa_aprovacao_por_ano)

# Gerar um gráfico de linha para mostrar a evolução da taxa de aprovação ao longo dos anos
plt.figure(figsize=(10, 6))
sns.lineplot(x=taxa_aprovacao_por_ano.index, y=taxa_aprovacao_por_ano.values, marker='o', color='blue')
plt.title('Evolução da Taxa de Aprovação ao Longo dos Anos')
plt.xlabel('Ano de Conclusão')
plt.ylabel('Taxa de Aprovação')
plt.show()

## 4.5 Perfil dos Alunos Assistidos pela ONG

In [ ]:
# Gerar estatísticas descritivas básicas para o perfil dos alunos assistidos pela ONG
perfil_alunos = df_merged[['IdAluno', 'IdResponsavelFinanceiro', 'QuantidadeFaltasAnual']].describe()

# Exibir estatísticas descritivas do perfil dos alunos
print(perfil_alunos)

# Verificar a distribuição dos alunos por IdResponsavelFinanceiro (indicador socioeconômico)
plt.figure(figsize=(10, 6))
sns.histplot(df_merged['IdResponsavelFinanceiro'].dropna(), bins=30, kde=False, color='lightblue')
plt.title('Distribuição Socioeconômica dos Alunos (IdResponsavelFinanceiro)')
plt.xlabel('IdResponsavelFinanceiro')
plt.ylabel('Contagem de Alunos')
plt.show()

# Verificar a distribuição de faltas
plt.figure(figsize=(10, 6))
sns.histplot(df_merged['QuantidadeFaltasAnual'].dropna(), bins=30, kde=True, color='lightgreen')
plt.title('Distribuição de Faltas Anuais dos Alunos')
plt.xlabel('Faltas Anuais')
plt.ylabel('Contagem de Alunos')
plt.show()
